In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import time
import warnings
import numpy as np
from nltk.corpus import stopwords
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
warnings.filterwarnings("ignore")
import sys
import os 
from tqdm import tqdm
import spacy

In [3]:
df = pd.read_csv('train.csv')
df['question1'] = df['question1'].apply(lambda x: str(x))
df['question2'] = df['question2'].apply(lambda x: str(x))

df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [7]:
questions = list(df['question1'])+list(df['question2'])
#find the tfidf scores after merging all the questions (to get the entire corpus)

tfidf = TfidfVectorizer(lowercase = False)
tfidf.fit_transform(questions)

#create dictionary with word:tfidf-score 
word2tfidf = dict(zip(tfidf.get_feature_names_out(), tfidf.idf_))

We find the importance of words using tfidf scores. Then use Glove, which derives relationship  between words from statistics, and weight normalize it using our tf-idf scores.

In [8]:
np.zeros([2,2])

array([[0., 0.],
       [0., 0.]])

In [38]:
nlp = spacy.load('en_core_web_sm')

vecs1 = [] # a vector representing every question is stored here 

for q1 in tqdm(list(df['question1'])):
    doc1 = nlp(q1)
    mean_vec1 = np.zeros([len(doc1), len(doc1[0].vector)])# array of no.of words (by) number of elements in vec representation
    for i,word1 in enumerate(doc1):
        # word2vec
        vec1 = word1.vector
        # get idf score
        try:
            idf = word2tfidf[str(word1)]
        except:
            idf = 0
        mean_vec1[i] += vec1*idf
    mean_vec1_m = mean_vec1.mean(axis = 0)
    vecs1.append(mean_vec1_m)
df['q1_feats_m'] = list(vecs1)    



100%|█████████████████████████████████████████████████████████████████████████| 404290/404290 [39:11<00:00, 171.95it/s]


In [39]:
vecs2 = [] # a vector representing every question is stored here 

for q2 in tqdm(list(df['question2'])):
    doc2 = nlp(q2)
    mean_vec2 = np.zeros([len(doc2), len(doc2[0].vector)])# array of no.of words (by) number of elements in vec representation
    for i,word2 in enumerate(doc2):
        # word2vec
        vec2 = word2.vector
        # get idf score
        try:
            idf = word2tfidf[str(word2)]
        except:
            idf = 0
        mean_vec2[i] += vec2*idf
    mean_vec2_m = mean_vec2.mean(axis = 0)
    vecs2.append(mean_vec2_m)
df['q2_feats_m'] = list(vecs2)    


100%|█████████████████████████████████████████████████████████████████████████| 404290/404290 [38:28<00:00, 175.15it/s]


In [40]:
# We import simple preprocessing features and,
# NLP features 
dfppro = pd.read_csv("df_fe_without_preprocessing_train.csv",encoding='latin-1')
dfnlp = pd.read_csv("nlp_features_train.csv",encoding='latin-1')

In [41]:
df1 = dfnlp.drop(['qid1','qid2','question1','question2'],axis=1)
df2 = dfppro.drop(['qid1','qid2','question1','question2','is_duplicate'],axis=1)
df3 = df.drop(['qid1','qid2','question1','question2','is_duplicate'],axis=1)
df3_q1 = pd.DataFrame(df3.q1_feats_m.values.tolist(), index= df3.index)
df3_q2 = pd.DataFrame(df3.q2_feats_m.values.tolist(), index= df3.index)    


In [42]:
# dataframe of nlp features
df1.head()

,id,is_duplicate,cwc_min,cwc_max,csc_min,csc_max,ctc_min,ctc_max,last_word_eq,first_word_eq,abs_len_diff,mean_len,token_set_ratio,token_sort_ratio,fuzz_ratio,fuzz_partial_ratio,longest_substr_ratio
0,0,0,0.999980,0.833319,0.999983,0.999983,0.916659,0.785709,0.0,1.0,2.0,13.0,100,93,93,100,0.982759
1,1,0,0.799984,0.399996,0.749981,0.599988,0.699993,0.466664,0.0,1.0,5.0,12.5,86,63,66,75,0.596154
2,2,0,0.399992,0.333328,0.399992,0.249997,0.399996,0.285712,0.0,1.0,4.0,12.0,63,63,43,47,0.166667
3,3,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,2.0,12.0,28,24,9,14,0.039216
4,4,0,0.399992,0.199998,0.999950,0.666644,0.571420,0.307690,0.0,1.0,6.0,10.0,67,47,35,56,0.175000


In [43]:
# data before preprocessing 
df2.head()

,id,freq_qid1,freq_qid2,q1len,q2len,q1_n_words,q2_n_words,word_Common,word_Total,word_share,freq_q1+q2,freq_q1-q2
0,0,1,1,66,57,14,12,10,13,0.769231,2,0
1,1,4,1,51,88,8,13,4,16,0.250000,5,3
2,2,1,1,73,59,14,10,4,20,0.200000,2,0
3,3,1,1,50,65,11,9,0,19,0.000000,2,0
4,4,3,1,76,39,13,7,2,18,0.111111,4,2


In [44]:
# Questions 1 tfidf weighted word2vec
df3_q1.head()

,0,1,2,3,4,5,6,7,8,9,...,86,87,88,89,90,91,92,93,94,95
0,1.900345,-0.444570,1.862308,-2.353578,-0.906011,0.187590,-2.276122,-2.648259,0.927211,0.509517,...,1.428383,-0.105255,-1.739925,0.961351,-2.912654,-3.476978,2.644579,4.347723,0.243254,0.142770
1,5.394716,0.968167,0.583148,-0.708473,-2.190906,2.843192,-3.287391,-0.307717,1.625830,-0.687556,...,0.635714,-0.651710,-4.490296,0.525582,0.486323,-3.079032,-1.238047,3.177230,1.892359,1.760751
2,1.086936,0.697746,-1.074795,-0.135651,-0.844778,-0.221674,-0.330972,-2.720161,1.293032,0.030586,...,2.433372,0.962446,0.388557,2.434410,-2.078901,-2.260784,-0.034031,-1.202895,0.430799,-0.584217
3,1.956699,4.653569,-1.441126,-1.357578,0.815954,2.635925,-0.587741,-1.466658,2.828143,-1.407127,...,-1.378428,1.381508,-0.876242,0.474220,0.107843,0.784156,2.501708,-1.559854,2.840963,-0.961133
4,5.603745,-2.098189,2.044607,-2.001144,2.404904,0.922884,-0.991817,-4.538134,5.389968,-0.318477,...,-2.717223,0.101938,-1.379425,2.204648,2.181839,-2.779178,0.890998,1.867777,2.034336,3.082953


In [45]:
print("Number of features in nlp dataframe :", df1.shape[1])
print("Number of features in preprocessed dataframe :", df2.shape[1])
print("Number of features in question1 w2v  dataframe :", df3_q1.shape[1])
print("Number of features in question2 w2v  dataframe :", df3_q2.shape[1])
print("Number of features in final dataframe  :", df1.shape[1]+df2.shape[1]+df3_q1.shape[1]+df3_q2.shape[1])

Number of features in nlp dataframe : 17
Number of features in preprocessed dataframe : 12
Number of features in question1 w2v  dataframe : 96
Number of features in question2 w2v  dataframe : 96
Number of features in final dataframe  : 221


In [46]:
#saving the data fram
df3_q1['id']=df1['id']
df3_q2['id']=df1['id']
df1  = df1.merge(df2, on='id',how='left')
df2  = df3_q1.merge(df3_q2, on='id',how='left')
result  = df1.merge(df2, on='id',how='left')
result.to_csv('final_features.csv')